In [119]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops


def detection_function(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    size_filter = 50
    nr,nc = image.shape[:2]

    offset = 25
    x_filter = int((nr)/offset)
    y_filter = int((nc)/offset)

    #塗色框
    rectangle=np.array([[1,2,3,4,0]])

    std= np.zeros((x_filter ,y_filter))
    co_correlation2 = np.zeros((x_filter ,y_filter))

    #條狀瑕疵變數
    judge=np.zeros([x_filter,y_filter],dtype=bool)
    countl=np.zeros(y_filter,dtype=int)
    decide=np.zeros(y_filter,dtype=bool)
    #洞洞瑕疵變數
    cor2abs=np.zeros((x_filter ,y_filter),dtype=bool)
    countt=np.zeros(y_filter,dtype=int)

    end_r = nr - size_filter + offset
    end_c = nc - size_filter + offset

    for i in range(0, end_r, offset):
        for j in range(0, end_c, offset):
            f = image[ i : i + size_filter , j : j + size_filter ]
       
            (meann , stdd) = cv2.meanStdDev(f)
            std[int(i/offset),int(j/offset)]=stdd
        
            result2 = greycomatrix( f, [2],[0] )
            co_correlation2[int(i/offset),int(j/offset)] = float(greycoprops(result2, 'correlation'))


    #相鄰差值計算
    for x in range(x_filter):
        for y in range(1,y_filter):
            #條狀瑕疵
            if(abs(std[x][y]-std[x][y-1])>1 and abs(std[x][y]-std[x][y-1])<2.2):
                judge[x][y]=True
                countl[y]=countl[y]+1
                if(countl[y]>=3):
                    decide[y]=True
                
            #洞洞瑕疵
            if(abs(co_correlation2[x][y]-co_correlation2[x][y-1])>=0.1):
                cor2abs[x][y]=True
                countt[y]=countt[y]+1
            
            
    #洞洞瑕疵 計算
    top=-1
    down=-1
    right=-1
    left=-1
    count=0
    for y in range(1,y_filter-1):
        for x in range(x_filter):
            if(cor2abs[x][y]==True):
                count=count+1
                if(left==-1 or y<left):
                    left=y
                if(top==-1 or x<top):
                    top=x
                if(down==-1 or x>down):
                    down=x
                if(right==-1 or y>right):
                    right=y
            if((countt[y-1]>0 or countt[y]>0) and x==x_filter-1):
                if(y==y_filter-3):
                    if(countt[y+1]==0):
                        right=y
                        if(left==right):
                            right=y
                            if(count>=(right-left+1)*(down-top+1)/2 and not(left==right and top==down)):
                                #cv2.rectangle(image,(right*offset-50 , top*offset+25 ),(right*offset , down*offset ) , (255,0,0) , 2)
                                add=np.array([[right*offset-50,top*offset+25,right*offset,down*offset,0]])
                                rectangle=np.append(rectangle,add ,axis=0)
                        elif(count>=(right-left+1)*(down-top+1)/2 and not(left==right and top==down)):
                            #cv2.rectangle(image,(left*offset+15 , top*offset ),(right*offset+50 , down*offset ) , (255,0,0) , 2) 
                            add=np.array([[left*offset+15,top*offset+25,right*offset+50,down*offset,0]])
                            rectangle=np.append(rectangle,add ,axis=0)
                        top=-1
                        down=-1
                        right=-1
                        left=-1
                        count=0
                elif(y==y_filter-2):
                    if(left==right):
                        right=y
                        if(count>=(right-left+1)*(down-top+1)/2 and not(left==right and top==down)):
                            #cv2.rectangle(image,(nc-40 , top*offset+25 ),(nc , down*offset ) , (255,0,0) , 2)
                            add=np.array([[nc-40,top*offset+25,nc,down*offset,0]])
                            rectangle=np.append(rectangle,add ,axis=0)
                    elif(count>=(right-left+1)*(down-top+1)/2 and not(left==right and top==down)):
                        #cv2.rectangle(image,(left*offset+25 , top*offset ),(right*offset+25 , down*offset+25 ) , (255,0,0) , 2)
                        add=np.array([[left*offset+25,top*offset,right*offset+25,down*offset+25,0]])
                        rectangle=np.append(rectangle,add ,axis=0)
                    top=-1
                    down=-1
                    right=-1
                    left=-1
                    count=0
                elif(countt[y]>0 and countt[y+1]==0 and countt[y+2]==0):
                    if(left==right):
                        right=y
                        if(count>=(right-left+1)*(down-top+1)/2 and not(left==right and top==down)):
                            #cv2.rectangle(image,(right*offset-50 , top*offset+25 ),(right*offset , down*offset ) , (255,0,0) , 2)
                            add=np.array([[right*offset-50,top*offset+25,right*offset,down*offset,0]])
                            rectangle=np.append(rectangle,add ,axis=0)
                        top=-1
                        down=-1
                        right=-1
                        left=-1
                        count=0
                    else:
                        right=y
                        if(count>=(right-left+1)*(down-top+1)/2 and not(left==right and top==down)):
                            #cv2.rectangle(image,(left*offset+15 , top*offset+30 ),(right*offset , down*offset+15 ) , (255,0,0) , 2)
                            add=np.array([[left*offset+15,top*offset+30,right*offset,down*offset+15,0]])
                            rectangle=np.append(rectangle,add ,axis=0)
                        top=-1
                        down=-1
                        right=-1
                        left=-1
                        count=0
                
    #洞洞瑕疵 計算end
            
    

    #條狀瑕疵 計算          
    for j in range (y_filter):
        if(decide[j]==True):
            if(j+1<y_filter and decide[j+1]==False):
                if(j+2<y_filter and decide[j+2]==True):
                    decide[j+1]==True

    leftl=nc
    rightl=0
    for j in range(y_filter):
        if(decide[j]==True):
            if(j+2<y_filter and decide[j+1]==False and decide[j+2]==False):
                if(leftl!=nc):
                    rightl=j
                    #cv2.rectangle(image,(leftl*offset-25 , 0 ),(rightl*offset+50 , nr ) , (255,0,0) , 2)
                    add=np.array([[leftl*offset,0,rightl*offset+40,nr,1]])
                    rectangle=np.append(rectangle,add ,axis=0)
                    leftl=nc
                    rightl=0
                else:
                    if(countl[j]==3):
                        if(countl[j-1]>0 or countl[j+1]>0):
                            #cv2.rectangle(image,(j*offset-25 , 0 ),((j+1)*offset+75 , nr ) , (255,0,0) , 2)
                            add=np.array([[j*offset-25,0,(j+1)*offset+75,nr,1]])
                            rectangle=np.append(rectangle,add ,axis=0)
                    else:
                        #cv2.rectangle(image,(j*offset-50 , 0 ),((j+1)*offset+50 , nr ) , (255,0,0) , 2)
                        add=np.array([[j*offset-25,0,(j+1)*offset+50,nr,1]])
                        rectangle=np.append(rectangle,add ,axis=0)
            elif(leftl==nc):
                leftl=j
            elif(j+2==y_filter):
                if(decide[j+1]==True):
                    rightl=j+1
                else:
                    rightl=j
                #cv2.rectangle(image,(leftl*offset-25 , 0 ),(rightl*offset+50 , nr ) , (255,0,0) , 2)
                add=np.array([[leftl*offset,0,rightl*offset+50,nr,1]])
                rectangle=np.append(rectangle,add ,axis=0)
    #條狀瑕疵 計算end
    return rectangle
    

def draw(rectangle,imgout):
    shaperec=rectangle.shape[0]
    for i in (1,shaperec-1,1):
        if(rectangle[i][4]==0):
            cv2.rectangle( imgout, ( rectangle[i][0], rectangle[i][1] ), ( rectangle[i][2], rectangle[i][3] ), ( 0, 0, 0 ), -1 )
        elif(rectangle[i][4]==1):#解決洞洞瑕疵與條條瑕疵 衝突
            contain=False
            for j in range(i-1,0,-1):
                if(rectangle[j][4]==0):
                    if(rectangle[i][0]<=rectangle[j][0]and rectangle[i][2]>=rectangle[j][2]):
                        contain=True
                    elif(rectangle[i][0]>=rectangle[j][0]and rectangle[i][2]>=rectangle[j][2]):
                        a=rectangle[i][0]
                        b=rectangle[j][2]
                        if(((b-a)/(rectangle[j][2]-rectangle[j][0]))>0.5):
                            contain=True
                    elif(rectangle[i][0]>=rectangle[j][0]and rectangle[i][2]<=rectangle[j][2]):
                        a=rectangle[j][0]
                        b=rectangle[i][2]
                        if(((b-a)/(rectangle[j][2]-rectangle[j][0]))>0.5):
                            contain=True
            if(not contain):
                cv2.rectangle( imgout, ( rectangle[i][0], rectangle[i][1] ), ( rectangle[i][2], rectangle[i][3] ), ( 0, 0, 0 ), -1 )
    return imgout
    
#主程式

#輸入檔名
image = cv2.imread('img1.jpg')
nr,nc = image.shape[:2]
imgout = np.ones( [ nr, nc, 3 ], dtype = 'uint8' )
cv2.rectangle( imgout, ( 0, 0 ), ( nc, nr ), ( 255, 255, 255 ), -1 )
rec=detection_function(image)
if(rec.shape[0]==1):
    image90=cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    rec2=detection_function(image90)
    if(rec2.shape[0]==1):
        print("沒有偵測到")
        cv2.imwrite( "result.png", imgout)
    else:
        print("有偵測到")
        imgout90=cv2.rotate(imgout, cv2.ROTATE_90_CLOCKWISE)
        imgout90draw=draw(rec2,imgout90)
        #cv2.imwrite( "resultt.png", imgout90draw)
        imglast=cv2.rotate(imgout90draw, cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.imwrite( "result.png", imglast)
else:
    print("有偵測到")
    imglast=draw(rec,imgout)
    cv2.imwrite( "result.png", imglast)
    

沒有偵測到
